In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime
import os
import warnings

from summer.utils import ref_times_to_dti

from autumn.core.project.project import get_project

from autumn.core.plots.utils import REF_DATE
from autumn.core.runs.managed import ManagedRun
from autumn.core.utils.pandas import pdfilt
from autumn.settings.region import Region
from autumn.core.plots.utils import get_plot_text_dict, change_xaxis_to_date, REF_DATE, ALPHAS, COLORS, _apply_transparency, _plot_targets_to_axis
from autumn.calibration.utils import get_uncertainty_df



In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
run_id = "sm_sir/malaysia/1670479022/e2df111"
region = "malaysia"

In [ ]:
mr = ManagedRun(run_id)

In [ ]:
full_run = mr.full_run.get_derived_outputs()
pbi = mr.powerbi.get_db()
targets = pbi.get_targets()
results = pbi.get_uncertainty()
mcmc_params = mr.calibration.get_mcmc_params()


In [ ]:
scenarios = [0, 1, 2, 3, 4, 5, 6]
quantile = 0.025

In [ ]:
outputs = ["notifications","hospital_admissions", "hospital_occupancy", "icu_occupancy", "infection_deaths"]
age_groups = ["0", "5", "10", "15", "20", "25", "30", "35", "40", "45", "50", "55", "60", "65", "70", "75"]
output_name = []
for output in outputs + ["non_hosp_notifications"]:
    for agegroup in age_groups:
        appending_output_name = f"{output}Xagegroup_{agegroup}"
        output_name.append(appending_output_name)
        
requested_derived_outputs = outputs + output_name 


In [ ]:
results_df = pd.DataFrame()

for scenario in scenarios:
    for i, output in enumerate(requested_derived_outputs):
        results_df[output] = results[(output, scenario, quantile)]
   

    results_df = results_df['2021-04-01':'2022-08-27']

    results_df['date_value'] = pd.to_datetime(results_df.index)
    results_df = results_df.set_index('date_value') 

#   converting to monthly sum
    results_df.resample('MS').sum()

    results_df.to_csv(f"C:/Users/pjay0011/Desktop/scenario_{scenario}.csv")


## Getting the basline and scenarios corresponding to a particular set of parameters

### for the sensitivity analsysis (abs_def_output)

In [ ]:
scenarios = [0, 1, 2, 3, 4, 5, 6]
sensitivity_outputs = ["scenario", "chain", "run", "times", "notifications", "hospital_occupancy",
                                  "infection_deaths", "icu_occupancy" ]
for scenario in scenarios:
    differnece_outputs = full_run[sensitivity_outputs]
    required_outputs = differnece_outputs.loc[(differnece_outputs["scenario"] == scenario)]
    required_outputs = required_outputs.iloc[: , 1:]
    required_outputs = required_outputs.drop(required_outputs.index[required_outputs["times"] < 457])  # 457-  April 01, 2021
    required_outputs["dates"] = (ref_times_to_dti(REF_DATE,required_outputs["times"]))
    required_outputs = required_outputs.drop("times", axis = 1)
    
    required_outputs.dates = pd.to_datetime(required_outputs.dates)
    required_outputs.set_index("dates", inplace=True)
    required_outputs.resample('MS').sum()
    

    required_outputs.to_csv(f"C:/Users/pjay0011/Desktop/sensitivity_scenario_{scenario}.csv")


